In [1]:
import re
import pandas as pd
import numpy as np
import sqlite3 
import sqlalchemy

from sqlalchemy import create_engine
from pathlib import Path
from datetime import datetime

In [2]:
engine = create_engine("sqlite:///registros.db", echo=False)

In [3]:
engine.execute("SELECT max(datetime) FROM registros").fetchall()

[('2020-02-17 11:32:00.000000',)]

In [4]:
result = engine.execute("SELECT max(sequential_number) FROM registros").fetchall()

if not (result and isinstance(result[0], sqlalchemy.engine.row.LegacyRow)):
    raise ValueError()

max_sequential_number = result[0][0]

print('max_sequential number = ', max_sequential_number)

max_sequential number =  43957


In [5]:
data_folder = Path('..', 'data')

data_files = list(data_folder.glob("*"))

data_files

[WindowsPath('../data/AFD00009000240006234 (1).txt'),
 WindowsPath('../data/AFD00009000240006234 (2).txt'),
 WindowsPath('../data/AFD00009000240006234.txt'),
 WindowsPath('../data/feriados_nacionais.xlsx')]

In [6]:
data_files[2]

WindowsPath('../data/AFD00009000240006234.txt')

In [7]:
pattern = re.compile("(\d{9})(\d{1})(\d{2})(\d{2})(\d{4})(\d{2})(\d{2})(\w{0,1})(\d{12})([\D]+)")

In [8]:
filename = data_files[2]
print(filename)

lines = None
count_lines = 0
count_register = 0

data = list()

with open(filename, 'r', encoding='ISO8859-1') as file:
    for line in file:
        count_lines += 1
        match = pattern.fullmatch(line)
        if match:
            count_register += 1
            register = match.groups()
            if int(register[0]) <= max_sequential_number:
                continue
            data.append(register)
        else:
            print(line)
        
    
print(count_lines, count_register)

columns = [
    'sequential_number',
    'cod_op',
    'day',
    'month',
    'year',
    'hour',
    'minutes',
    'cod_input',
    'pis',
    'name'
]

frame = pd.DataFrame(data, columns=columns)

del data

# frame['name'] = frame['name'].str.replace('\ {0,}\n{1,}', '')
frame['name'] = frame.name.str.strip('[ \n]')


frame[['name', 'cod_input']] = frame[['name', 'cod_input']].replace('', np.nan)

numeric_attribs = ['sequential_number', 'cod_op', 'day', 'month', 'year', 'hour', 'minutes' ]

frame[numeric_attribs] = frame[numeric_attribs].astype(int)

date_attribs = [ 'year', 'month', 'day', 'hour', 'minutes']

frame['datetime']  = pd.to_datetime(frame[date_attribs])

..\data\AFD00009000240006234.txt
0000000001177778744000166000000000000CÂMARA MUNICIPAL DE SANTO ANTONIO DA PLATINA                                                                                                          000090002400062341809201419072022190720220926

0000000012180920141115177778744000166000000000000CÂMARA MUNICIPAL DE SANTO ANTONIO DA PLATINA                                                                                                          CAMARA MUNICIPAL DE ST. ANT. DA PLATINA                                                             

0000210372200320171315177778744000166000000000000SANTO ANTONIO DA PLATINA CAMARA DE VEREADORES                                                                                                         CÂMARA MUNICIPAL DE SANTO ANTÔNIO DA PLATINA - AVENIDA CORONEL OLIVEIRA MOTTA, 715 - CENTRO         

0000210522210320170816177778744000166000000000000CÂMARA MUNICIPAL DE SANTO ANTONIO DA PLATINA                                      

In [9]:
print(frame.shape)
frame.head()

(17378, 11)


,sequential_number,cod_op,day,month,year,hour,minutes,cod_input,pis,name,datetime
0,43958,3,17,2,2020,12,0,NaN,016002684248,NaN,2020-02-17 12:00:00
1,43959,3,17,2,2020,12,0,NaN,020071346877,NaN,2020-02-17 12:00:00
2,43960,3,17,2,2020,12,1,NaN,012699519512,NaN,2020-02-17 12:01:00
3,43961,3,17,2,2020,12,1,NaN,013118171501,NaN,2020-02-17 12:01:00
4,43962,3,17,2,2020,12,1,NaN,013979783277,NaN,2020-02-17 12:01:00


In [10]:
with engine.begin() as connection:
    frame.to_sql('registros', 
                     con=connection, 
                     if_exists='append',
                     index=False)

In [11]:
engine.execute("SELECT max(sequential_number) FROM registros").fetchall()

[(61335,)]

In [12]:
engine.execute("SELECT max(datetime) FROM registros").fetchall()

[('2022-07-19 08:00:00.000000',)]